In [ ]:
import sys
sys.path.append('../')
from opal import SourceBasic, StageNonlinear1D, InterstageELEGANT, BeamDeliverySystemBasic, Linac, Collider, InteractionPointGUINEAPIG
from opal.utilities.plasmaphysics import beta_matched
import numpy as np


# define stage
stage = StageNonlinear1D()
stage.deltaE = 5.5e9 # [eV]
stage.L = 0.8 # [m]
stage.n0 = 1e22 # [m^-3]
stage.kRb = 2.5

# define beam
source = SourceBasic()
source.L = 0.5 # [m]
source.Q = -1e-9 # [C]
source.E0 = 3e9 # [eV]
source.sigE = 0.01*source.E0 # [eV]
source.sigz = 8e-6 # [m]
source.z = -79.5e-6 # [m]
source.emitnx = 1e-6 # [m rad]
source.emitny = 0.03e-6 # [m rad]
source.betax = beta_matched(stage.n0, source.E0) # [m]
source.betay = source.betax
source.Npart = 2000

# define interstage
interstage = InterstageELEGANT()
interstage.beta0 = lambda E: beta_matched(stage.n0, E) # [T]
interstage.Bdip = 0.5 # [T]
interstage.Ldip = lambda E: 0.8 * np.sqrt(E/10e9) # [m(eV)]

# beam delivery system
bds = BeamDeliverySystemBasic()
bds.beta_waist_x = 8e-3 # [m]
bds.beta_waist_y = 0.4e-3 # [m]
bds.L = 20

# define linac
linac = Linac(source, stage, interstage, bds, 14)

# define interaction point
ip = InteractionPointGUINEAPIG()

# define collider
collider = Collider(linac, bds, ip)

# track particles
collider.run("gamma-gamma", shots=10, overwrite=False)

# plot survery and beam evolution
stage.plotWakefield(source.track())
linac.plotSurvey()
linac.plotEvolution()
linac.plotWaterfalls()

>> LINAC #1
>> SHOT #1
Tracking element #1 (s = 0.5 m, SourceBasic, stage 0)


/Users/carlal/Library/jupyterlab-desktop/jlab_server/lib/python3.8/site-packages/scipy/integrate/_odepack_py.py:247: ODEintWarning: Excess work done on this call (perhaps wrong Dfun type). Run with full_output = 1 to get quantitative information.
  warnings.warn(warning_msg, ODEintWarning)


Tracking element #2 (s = 1.3 m, StageNonlinear1D, stage 1)
Tracking element #3 (s = 4.83 m, InterstageELEGANT, stage 1)


/Users/carlal/Library/jupyterlab-desktop/jlab_server/lib/python3.8/site-packages/scipy/integrate/_odepack_py.py:247: ODEintWarning: Excess work done on this call (perhaps wrong Dfun type). Run with full_output = 1 to get quantitative information.
  warnings.warn(warning_msg, ODEintWarning)


Tracking element #4 (s = 5.63 m, StageNonlinear1D, stage 2)
Tracking element #5 (s = 10.17 m, InterstageELEGANT, stage 2)


...

/Users/carlal/Library/jupyterlab-desktop/jlab_server/lib/python3.8/site-packages/scipy/integrate/_odepack_py.py:247: ODEintWarning: Excess work done on this call (perhaps wrong Dfun type). Run with full_output = 1 to get quantitative information.
  warnings.warn(warning_msg, ODEintWarning)


Tracking element #20 (s = 66.83 m, StageNonlinear1D, stage 10)
Tracking element #21 (s = 76.06 m, InterstageELEGANT, stage 10)


/Users/carlal/Library/jupyterlab-desktop/jlab_server/lib/python3.8/site-packages/scipy/integrate/_odepack_py.py:247: ODEintWarning: Excess work done on this call (perhaps wrong Dfun type). Run with full_output = 1 to get quantitative information.
  warnings.warn(warning_msg, ODEintWarning)


Tracking element #22 (s = 76.86 m, StageNonlinear1D, stage 11)
Tracking element #23 (s = 86.51 m, InterstageELEGANT, stage 11)


/Users/carlal/Library/jupyterlab-desktop/jlab_server/lib/python3.8/site-packages/scipy/integrate/_odepack_py.py:247: ODEintWarning: Excess work done on this call (perhaps wrong Dfun type). Run with full_output = 1 to get quantitative information.
  warnings.warn(warning_msg, ODEintWarning)
